In [55]:
import pandas as pd

In [56]:
df = pd.read_csv("../../preprocessed_data/[10분 단위]아파트_층별_소비전력_2023-03-16 06.30.00 ~ 2023-08-30 10.30.00.csv", usecols=[1,2,3,4])

df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588818 entries, 0 to 1588817
Data columns (total 4 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   TIMESTAMP       1588818 non-null  datetime64[ns]
 1   BUILDING        1588818 non-null  int64         
 2   FLOOR           1588818 non-null  int64         
 3   CONSUMPTION(W)  1588818 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 48.5 MB


In [57]:
df

,TIMESTAMP,BUILDING,FLOOR,CONSUMPTION(W)
0,2023-03-16 06:30:00,561,1,225
1,2023-03-16 06:40:00,561,1,217
2,2023-03-16 06:50:00,561,1,212
3,2023-03-16 07:00:00,561,1,88
4,2023-03-16 07:10:00,561,1,435
...,...,...,...,...
1588813,2023-08-30 09:50:00,563,24,366
1588814,2023-08-30 10:00:00,563,24,545
1588815,2023-08-30 10:10:00,563,24,567
1588816,2023-08-30 10:20:00,563,24,547


In [58]:
df.query("`BUILDING` == 561 and `FLOOR` == 1")['CONSUMPTION(W)']

0        225
1        217
2        212
3         88
4        435
        ... 
24068    354
24069    354
24070    387
24071    505
24072    547
Name: CONSUMPTION(W), Length: 24073, dtype: int64

In [59]:
df_561_1 = df.query("`BUILDING` == 561 and `FLOOR` == 1")
df_561_1['CONSUMPTION(W)'] += df.query("`BUILDING` == 561 and `FLOOR` == 2")['CONSUMPTION(W)'].to_list()

# df_561_1['CONSUMPTION3'] = df_561_1['CONSUMPTION2'] + df_561_1['CONSUMPTION(W)']
df_561_1

C:\Users\hansung\AppData\Local\Temp\ipykernel_23928\1651499509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_561_1['CONSUMPTION(W)'] += df.query("`BUILDING` == 561 and `FLOOR` == 2")['CONSUMPTION(W)'].to_list()


,TIMESTAMP,BUILDING,FLOOR,CONSUMPTION(W)
0,2023-03-16 06:30:00,561,1,481
1,2023-03-16 06:40:00,561,1,469
2,2023-03-16 06:50:00,561,1,449
3,2023-03-16 07:00:00,561,1,395
4,2023-03-16 07:10:00,561,1,739
...,...,...,...,...
24068,2023-08-30 09:50:00,561,1,1598
24069,2023-08-30 10:00:00,561,1,1663
24070,2023-08-30 10:10:00,561,1,1620
24071,2023-08-30 10:20:00,561,1,1356


In [60]:
df_561_1 = df.query("`BUILDING` == 561 and `FLOOR` == 1")
df_561_2 = df.query("`BUILDING` == 561 and `FLOOR` == 2")


pd.concat([df_561_1,df_561_2])

,TIMESTAMP,BUILDING,FLOOR,CONSUMPTION(W)
0,2023-03-16 06:30:00,561,1,225
1,2023-03-16 06:40:00,561,1,217
2,2023-03-16 06:50:00,561,1,212
3,2023-03-16 07:00:00,561,1,88
4,2023-03-16 07:10:00,561,1,435
...,...,...,...,...
48141,2023-08-30 09:50:00,561,2,1244
48142,2023-08-30 10:00:00,561,2,1309
48143,2023-08-30 10:10:00,561,2,1233
48144,2023-08-30 10:20:00,561,2,851


In [61]:
final_df = pd.DataFrame({'TIMESTAMP':[], 'BUILDING':[],	'FLOOR':[],	'CONSUMPTION(W)':[]})

buildings = [561, 562, 563]
for building in buildings:
    floors = df.query("`BUILDING` == @building")['FLOOR'].drop_duplicates().sort_values()

    accu_df = None
    cnt = 0
    refloor = 1
    for floor in floors:
        now_df = df.query("`BUILDING` == @building and `FLOOR` == @floor")
        if cnt % 2 == 0:
            accu_df = now_df
        elif cnt % 2 == 1:
            accu_df['CONSUMPTION(W)'] += now_df['CONSUMPTION(W)'].to_list()
            accu_df['FLOOR'] = refloor
            refloor += 1

            # accu_df = accu_df.iloc[670:]
            # accu_df = accu_df.reset_index(drop=True)

            final_df = pd.concat([final_df, accu_df])
        
        cnt += 1


final_df
    

C:\Users\hansung\AppData\Local\Temp\ipykernel_23928\4211865040.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accu_df['CONSUMPTION(W)'] += now_df['CONSUMPTION(W)'].to_list()
C:\Users\hansung\AppData\Local\Temp\ipykernel_23928\4211865040.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accu_df['FLOOR'] = refloor
C:\Users\hansung\AppData\Local\Temp\ipykernel_23928\4211865040.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version,

,TIMESTAMP,BUILDING,FLOOR,CONSUMPTION(W)
0,2023-03-16 06:30:00,561.0,1.0,481.0
1,2023-03-16 06:40:00,561.0,1.0,469.0
2,2023-03-16 06:50:00,561.0,1.0,449.0
3,2023-03-16 07:00:00,561.0,1.0,395.0
4,2023-03-16 07:10:00,561.0,1.0,739.0
...,...,...,...,...
1564740,2023-08-30 09:50:00,563.0,12.0,488.0
1564741,2023-08-30 10:00:00,563.0,12.0,672.0
1564742,2023-08-30 10:10:00,563.0,12.0,688.0
1564743,2023-08-30 10:20:00,563.0,12.0,625.0


In [62]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 794409 entries, 0 to 1564744
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   TIMESTAMP       794409 non-null  datetime64[ns]
 1   BUILDING        794409 non-null  float64       
 2   FLOOR           794409 non-null  float64       
 3   CONSUMPTION(W)  794409 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 30.3 MB


In [63]:
final_df = final_df.astype({"BUILDING": int, "FLOOR": int, 'CONSUMPTION(W)':int})

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 794409 entries, 0 to 1564744
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   TIMESTAMP       794409 non-null  datetime64[ns]
 1   BUILDING        794409 non-null  int32         
 2   FLOOR           794409 non-null  int32         
 3   CONSUMPTION(W)  794409 non-null  int32         
dtypes: datetime64[ns](1), int32(3)
memory usage: 21.2 MB


In [64]:
final_df.to_csv("../../preprocessed_data/[AI]아파트_층별_소비전력_2023-03-16 06.30.00 ~ 2023-08-30 10.30.00.csv")
# final_df.to_csv("../../preprocessed_data/[DB]아파트_층별_소비전력_2023-03-16 06.30.00 ~ 2023-08-30 10.30.00.csv")